# Australian Geoscience Datacube
## Feature Summary Examples

The [Australian Geoscience Datacube](https://github.com/data-cube/agdc-v2) provides an integrated gridded data analysis environment for decades of analysis ready earth observation satellite and related data from multiple satellite and other acquisition systems.

For instructions on using the Datacube on NCI, see: http://agdc-v2.readthedocs.io/en/develop/nci_usage.html

For instructions on setting up your own instance, see: http://agdc-v2.readthedocs.io/en/develop/install.html

This notebook touches briefly on some the implimented features of the Datacube module, and is only intended to deomstrat functionality rather than be a tutorial.

In [1]:
%matplotlib inline
import datacube
import ogr
import xarray as xr
from datacube.storage import masking
import pandas


If you have set up your config correctly, or are using the module on NCI, you should be able to make `Datacube` object that can connects to the configured datacube system.

In [2]:
dc = datacube.Datacube(app='dc-example')
dc

Datacube<index=Index<db=PostgresDb<engine=Engine(postgresql://lxl554@130.56.244.227:6432/datacube)>>>

## Datacube products and measurements
The Datacube provides pandas.DataFrame representations of the available products and measurements:

In [3]:
#dc.list_products()

## Datacube Measurements
The list of measurements stored in the datacube can also be listed.

Measurements are also known as _bands_ in the imagery domain, and _data variables_ when stored in NetCDF files or when working with `xarray.Dataset` objects.

In [4]:
#dc.list_measurements()

In [5]:
#Use this if you want to specify a shapefile
#open shapefile and retrieve extents
vec_fname = '/g/data/r78/lxl554/hovmoller/Murray Mouth.shp'
vec_ds = ogr.Open(vec_fname)
layer = vec_ds.GetLayer(0)
extent = layer.GetExtent()
print "Extent: {}".format(extent)

Extent: (138.8742806106138, 138.8856287881116, -35.5589930957951, -35.55440732305348)


## Retrieve surface reflectance data


In [6]:
#### DEFINE SPATIOTEMPORAL RANGE AND BANDS OF INTEREST
#Use this to manually define an upper left/lower right coords
#Define spatial range
#lat_max = -27.75
lat_max = extent[3]
#lat_min = -27.85
lat_min = extent[2]
#lon_max = 137.57756
lon_max = extent[1]
#lon_min = 137.30746
lon_min = extent[0]

#Define temporal range
start_of_epoch = '1987-01-01'
#need a variable here that defines a rolling 'latest observation'
end_of_epoch =  '2016-01-31'

#Define wavelengths/bands of interest, remove this kwarg to retrieve all bands
bands_of_interest = ['nir','red']

#Define sensors of interest
sensor1 = 'ls8'
sensor2 = 'ls7'
sensor3 = 'ls5'


# retrieve the NBAR and PQ for the spatiotemporal range of interest


In [7]:
sensor1_nbar = dc.load(product= sensor1+'_nbar_albers', 
                   x=(lon_min, lon_max), y=(lat_max, lat_min), 
                   time = (start_of_epoch, end_of_epoch), 
                   measurements = bands_of_interest,
                   group_by='solar_day')

sensor1_pq = dc.load(product= sensor1+'_pq_albers', 
                   x=(lon_min, lon_max), y=(lat_max, lat_min), 
                   time = (start_of_epoch, end_of_epoch),
                   group_by='solar_day')
            

In [8]:
#This line exists to make sure that there's a 1:1 match between NBAR and PQ
sensor1_nbar = sensor1_nbar.sel(time = sensor1_pq.time)

In [9]:
sensor2_nbar = dc.load(product= sensor2+'_nbar_albers', 
                   x=(lon_min, lon_max), y=(lat_max, lat_min), 
                   time = (start_of_epoch, end_of_epoch), 
                   measurements = bands_of_interest,
                   group_by='solar_day')
                   
sensor2_pq = dc.load(product= sensor2+'_pq_albers', 
                   x=(lon_min, lon_max), y=(lat_max, lat_min), 
                   time = (start_of_epoch, end_of_epoch),
                   group_by='solar_day')
                   

In [11]:
sensor2_nbar = sensor2_nbar.sel(time = sensor2_pq.time)

In [12]:
sensor3_nbar = dc.load(product= sensor3+'_nbar_albers', 
                   x=(lon_min, lon_max), y=(lat_max, lat_min), 
                   time = (start_of_epoch, end_of_epoch), 
                   measurements = bands_of_interest,
                   group_by='solar_day')
                   
sensor3_pq = dc.load(product= sensor3+'_pq_albers', 
                   x=(lon_min, lon_max), y=(lat_max, lat_min), 
                   time = (start_of_epoch, end_of_epoch),
                   group_by='solar_day')

In [13]:
sensor3_nbar = sensor3_nbar.sel(time = sensor3_pq.time)

In [14]:

print len (sensor1_nbar.time)
print len (sensor1_pq.time)
print len (sensor2_nbar.time)
print len (sensor2_pq.time)
print len (sensor3_nbar.time)
print len (sensor3_pq.time)


60
60
208
208
212
212


In [15]:
#Concatenate NBAR from all sensors and sort by time and likewise for PQ
nbar = xr.concat([sensor1_nbar, sensor2_nbar, sensor3_nbar], dim='time')
time_sorted = nbar.time.argsort()
nbar_sorted = nbar.isel(time=time_sorted)

pq = xr.concat([sensor1_pq, sensor2_pq, sensor3_pq], dim='time')
pq_time_sorted = pq.time.argsort()
pq_sorted = pq.isel(time=pq_time_sorted)

In [16]:
print pq_sorted.dims
print nbar_sorted.dims

Frozen(SortedKeysDict({'y': 24, 'x': 43, 'time': 480}))
Frozen(SortedKeysDict({'y': 24, 'x': 43, 'time': 480}))


In [17]:
#This one works...
fmask_cloud_free = masking.make_mask(sensor1_pq, 
                              #cloud_acca='no_cloud', 
                              cloud_fmask='no_cloud',
                              #blue_saturated = False,
                              #green_saturated = False,
                              #red_saturated = False,
                              #nir_saturated = False,
                              #swir1_saturated = False,
                              #swir2_saturated = False,
                              contiguous=True)
nir = sensor1_nbar.nir.where(sensor1_nbar.nir != sensor1_nbar.nir.attrs['nodata'])
red = sensor1_nbar.red.where(sensor1_nbar.red != sensor1_nbar.red.attrs['nodata'])
fmask_good_data = fmask_cloud_free.pixelquality.loc[start_of_epoch:end_of_epoch]

ndvi_fmask = ((nir - red) / (nir + red)).where(fmask_good_data)

In [18]:
sensor1_pq

<xarray.Dataset>
Dimensions:       (time: 60, x: 43, y: 24)
Coordinates:
  * time          (time) datetime64[ns] 2013-04-20 2013-05-06 2013-05-22 ...
  * y             (y) float64 -3.9e+06 -3.901e+06 -3.901e+06 -3.901e+06 ...
  * x             (x) float64 6.221e+05 6.222e+05 6.222e+05 6.222e+05 ...
Data variables:
    pixelquality  (time, y, x) int16 16383 16383 16383 16383 16383 16383 ...
Attributes:
    crs: EPSG:3577

In [19]:
pq_sorted

<xarray.Dataset>
Dimensions:       (time: 480, x: 43, y: 24)
Coordinates:
  * y             (y) float64 -3.9e+06 -3.901e+06 -3.901e+06 -3.901e+06 ...
  * x             (x) float64 6.221e+05 6.222e+05 6.222e+05 6.222e+05 ...
  * time          (time) datetime64[ns] 1987-05-31 1987-09-20 1987-10-06 ...
Data variables:
    pixelquality  (time, y, x) int16 15359 15359 15359 15359 15359 15359 ...
Attributes:
    crs: EPSG:3577

In [22]:
#But this one doesn't??? Only difference I can see is that pq_sorted object now has 3 coordinates (now includes time)
good_data = masking.make_mask(pq_sorted, 
                              #cloud_acca='no_cloud', 
                              cloud_fmask='no_cloud',
                              #blue_saturated = False,
                              #green_saturated = False,
                              #red_saturated = False,
                              #nir_saturated = False,
                              #swir1_saturated = False,
                              #swir2_saturated = False,
                              #contiguous=True)
                              )
nir = nbar_sorted.nir.where(nbar_sorted.nir != nbar_sorted.nir.attrs['nodata'])
red = nbar_sorted.red.where(nbar_sorted.red != nbar_sorted.red.attrs['nodata'])
sorted_good_data = good_data.pixelquality.loc[start_of_epoch:end_of_epoch]

ndvi = ((nir - red) / (nir + red)).where(sorted_good_data)

ValueError: No masking variable found

In [ ]:
sensor1_nbar['red'].isel(x=[10], y=[10]).plot()

In [ ]:
nbar_sorted['red'].isel(x=[10], y=[10]).plot()

In [ ]:
pq_sorted['pixelquality'].isel(x=[10], y=[10])

In [ ]:
#len (sensor_combined.time)

The returned data is an `xarray.Dataset` object, which is a labelled n-dimensional array wrapping a `numpy` array.

We can investigate the data to see the variables (measurement bands) and dimensions that were returned:

In [ ]:
#to view the attributes of the xarray.data
#sensor1_pq
#ls7_nbar
#ls8_nbar

In [ ]:
#print sensor1_nbar.time.min()
print sensor1_nbar.time.min()
#print sensor2_nbar.time.min()
print sensor1_pq.time.max()

In [ ]:
def calculate_size(bytes, gb=False):
    size = bytes / 1024.0 / 1024.0
    if gb: 
        return size / 1024.0
    return size
print "Array size in Gb:{}".format(calculate_size(sensor1_nbar.nbytes, True))
VDIram = 12
RAMpercent = ((calculate_size(sensor1_nbar.nbytes, True)/VDIram)*100)
print "Selected array is " +str(RAMpercent)+ " percent of available RAM"

In [ ]:
#to view variables
sensor1_nbar.data_vars

## Masking out no_data and cloud affected pixels
When there is no data availaible, such as on the boundaries of a scene, it is filled in with a special value.
We can use filter it out, although xarray will convert the data from `int` to `float` so that it can use `NaN` to indicate no data.

Now that bad values are no longer represented as `-9999`, the data fits on a much better colour ramp:

In [ ]:
#help (sensor1_pq)

#pandas.DataFrame.from_dict(masking.get_flags_def(sensor1_pq), orient='index')

## Some basic band maths
We can combine the `red` and `nir` (_near-infrared_) bands to calculate NDVI (_normalised difference vegetation index_).

In [ ]:
acca_cloud_free = masking.make_mask(sensor1_pq, 
                              cloud_acca='no_cloud', 
                              #cloud_fmask='no_cloud',
                              #blue_saturated = False,
                              #green_saturated = False,
                              #red_saturated = False,
                              #nir_saturated = False,
                              #swir1_saturated = False,
                              #swir2_saturated = False,
                              contiguous=True)
nir = sensor1_nbar.nir.where(sensor1_nbar.nir != sensor1_nbar.nir.attrs['nodata'])
red = sensor1_nbar.red.where(sensor1_nbar.red != sensor1_nbar.red.attrs['nodata'])
acca_good_data = acca_cloud_free.pixelquality.loc[start_of_epoch:end_of_epoch]

ndvi_acca = ((nir - red) / (nir + red)).where(acca_good_data)

In [ ]:
saturation_free = masking.make_mask(sensor1_pq, 
                              #cloud_acca='no_cloud', 
                              #cloud_fmask='no_cloud',
                              blue_saturated = False,
                              green_saturated = False,
                              red_saturated = False,
                              nir_saturated = False,
                              swir1_saturated = False,
                              swir2_saturated = False,
                              contiguous=True)
nir = sensor1_nbar.nir.where(sensor1_nbar.nir != sensor1_nbar.nir.attrs['nodata'])
red = sensor1_nbar.red.where(sensor1_nbar.red != sensor1_nbar.red.attrs['nodata'])
saturation_good_data = saturation_free.pixelquality.loc[start_of_epoch:end_of_epoch]

ndvi_saturation = ((nir - red) / (nir + red)).where(saturation_good_data)

In [ ]:
contiguous = masking.make_mask(sensor1_pq, contiguous=True)

pq_contig = contiguous.pixelquality.loc[start_of_epoch:end_of_epoch]


## Pixel drill

In [ ]:
ndvi_fmask.isel(x=[10], y=[10]).plot()

In [ ]:
#mostly_good_ndvi.isel(y=100).plot()

## Plotting a multi-band image

In [ ]:
rgb = dc.load(product= sensor1+'_nbar_albers', 
              x=(lon_min, lon_max), y=(lat_max, lat_min), 
              time = (start_of_epoch, end_of_epoch), 
              measurements=['swir1', 'nir', 'green'], 
              group_by='solar_day', stack='color',
              dask_chunks={'time':1, 'x': 1000, 'y': 1000} ).transpose('time', 'y', 'x', 'color')
            

zip(rgb.dims, rgb.shape)

In [ ]:
fake_saturation = 6000
clipped_visible = rgb.where(rgb<fake_saturation).fillna(fake_saturation)
max_val = clipped_visible.max(['y', 'x'])
scaled = (clipped_visible / max_val)

In [ ]:
from matplotlib import pyplot as plt
fig, axes = plt.subplots(ncols=2, nrows = 2, figsize=(11.69,8.27))
t = 11
axes[0,0].imshow(scaled.isel(time = t))
axes[0,0].set_title(scaled.time[t])
axes[0,1].imshow(ndvi_acca.isel(time = t), cmap = 'viridis')
axes[0,1].set_title('ACCA')
axes[1,0].imshow(ndvi_fmask.isel(time = t), cmap = 'viridis')
axes[1,0].set_title('Fmask')
axes[1,1].imshow(ndvi_saturation.isel(time = t), cmap = 'viridis')
axes[1,1].set_title('Saturated')
#tight_layout()

In [ ]:
from matplotlib import pyplot as plt
fig, axes = plt.subplots(ncols=2, nrows = 2, figsize=(11.69,8.27))
t = 3
axes[0,0].imshow(scaled.isel(time = t))
axes[0,0].set_title(scaled.time[t])
axes[0,1].imshow(acca_good_data.sum('time')/pq_contig.sum('time').astype(float))
axes[0,1].set_title('ACCA')
axes[1,0].imshow(fmask_good_data.sum('time')/pq_contig.sum('time').astype(float))
axes[1,0].set_title('Fmask')
axes[1,1].imshow(saturation_good_data.sum('time')/pq_contig.sum('time').astype(float))
axes[1,1].set_title('Saturated')
#tight_layout()'''

In [ ]:
#fig,  figsize=(11.69,8.27)


saturation_good_data.sum('time').plot()

In [ ]:
acca_good_data.sum('time').plot()

In [ ]:
plt.imshow(ndvi_acca.isel(time = t), cmap = 'viridis')

In [ ]:
plt.imshow(ndvi_fmask.isel(time = t), cmap = 'viridis')

In [ ]:
plt.imshow(ndvi_saturation.isel(time = t), cmap = 'viridis')